# World Bank API

> The following is an example of a [Jupyter notebook](https://jupyter.org) - a tutorial of how to retrieve data from the [World Bank Indicators API](https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-about-the-indicators-api-documentation) - that illustrates how to use computational content with the [template](https://datapartnership.org/template). 

## Requirements

In [67]:
import itertools

import bokeh
import requests
import pandas

from bokeh.palettes import Spectral6
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import BasicTickFormatter

## Retrieve Data

In this example, we retrieve **Population, total** (`SP.POP.TOTL`) from the [World Bank Indicators](https://data.worldbank.org/indicator) for [BRICS](https://infobrics.org).

In [38]:
url = "https://api.worldbank.org/v2/country/chn;bra;ind;rus;zaf/indicator/SP.POP.TOTL?format=json&per_page=10000"

Let's use [requests](https://requests.readthedocs.io) to send a GET request,

In [40]:
r = requests.get(url)

Now, let's normalize and create `pandas.DataFrame` from the response,

In [47]:
# normalize
data = pandas.json_normalize(r.json()[-1])

df = pandas.DataFrame.from_dict(data)

```{tip}
Alternatively, the World Bank API supports downloading the data as an [archive](http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?date=2000&source=2&downloadformat=csv). 
```

Let's take a look at the dataframe, 

In [48]:
df = df.pivot_table(values="value", index="date", columns="countryiso3code")

Let's scale to the numbers to **millions**.

In [49]:
df = df / 1e6

countryiso3code,BRA,CHN,IND,RUS,ZAF
date,,,,,
1960,72179235,667070000,450547675,119897000,17099836
1961,74311338,660330000,459642166,121236000,17524533
1962,76514329,665770000,469077191,122591000,17965733
1963,78772647,682335000,478825602,123960000,18423157
1964,81064572,698355000,488848139,125345000,18896303
...,...,...,...,...,...
2017,207833825,1396215000,1338676779,144496739,57009751
2018,209469320,1402760000,1352642283,144477859,57792520
2019,211049519,1407745000,1366417756,144406261,58558267


## Visualization

In [56]:
output_notebook()

Loading BokehJS ...

In [68]:
# instantiating the figure object
p = figure(title = "Population, total (World Bank)", width=700, height=600)

# colors
colors = itertools.cycle(Spectral6) 

# plotting the line graph
for column, color in zip(df.columns, colors):
    p.line(df.index, df[column], legend_label=column, color=color, line_width=2,)

p.legend.location = "right"
p.legend.click_policy="mute"
p.title.text_font_size = "12pt"

p.xaxis.axis_label = "Year"
p.yaxis.axis_label = "Population, total (in millions)"

In [69]:
show(p)